In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [16]:
data = pd.read_csv("../data/football_matches.csv", dtype={'manager_pt1': str, 'manager_pt2': str})
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,pt1,pt2,score_pt1,score_pt2,ground,country,league,date,time,season,...,crosses_pt2,dribbles_completed_pt2,dribbles_pt2,possession_lost_pt2,duels_won_pt2,aerials_won_pt2,tackles_pt2,interceptions_pt2,clearances_pt2,attendance
0,palermo,lazio,5,1,Renzo Barbera,Italy,Italian Serie A,2012-02-19 19:45:00+00:00,19:45:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
1,novara,atalanta,0,0,Silvio Piola,NaN,Italian Serie A,2012-02-19 14:00:00+00:00,14:00:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
2,udinese,cagliari,0,0,Friuli,Italy,Italian Serie A,2012-02-19 19:45:00+00:00,19:45:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
3,lecce,siena,4,1,Via del Mare,Italy,Italian Serie A,2012-02-19 11:30:00+00:00,11:30:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
4,as_roma,parma,1,0,Olimpico,NaN,Italian Serie A,2012-02-19 14:00:00+00:00,14:00:00,2012,...,0,0,0,0,0,0,0,0,0,NaN


In [22]:
def calculate_elo_ratings(df, elo_ratings):
    # Calculate the expected scores for each match
    rating_1 = np.array([elo_ratings[team] for team in df['pt1']])
    rating_2 = np.array([elo_ratings[team] for team in df['pt2']])
    expected_scores_1 = 1 / (1 + 10 ** ((rating_2 - rating_1) / 400))
    expected_scores_2 = 1 / (1 + 10 ** ((rating_1 - rating_2) / 400))

    # Calculate the actual scores for each match
    actual_scores_1 = np.where(df['score_pt1'] > df['score_pt2'], 1, 0)
    actual_scores_2 = np.where(df['score_pt1'] < df['score_pt2'], 1, 0)

    # Calculate the new Elo ratings for each team
    k = 32
    new_ratings_1 = rating_1 + k * (actual_scores_1 - expected_scores_1)
    new_ratings_2 = rating_2 + k * (actual_scores_2 - expected_scores_2)

    # Update the Elo ratings in the dataframe
    for i in range(df.shape[0]):
        team_1 = df.iloc[i]['pt1']
        team_2 = df.iloc[i]['pt2']
        elo_ratings[team_1] = new_ratings_1[i]
        elo_ratings[team_2] = new_ratings_2[i]

    return df, elo_ratings


def init_elos(df):
    # Get list of unique teams
    teams_list = (df['pt1'].concat(df['pt2'])).unique()
    teams_dict = dict.fromkeys(teams_list, 1500)

    return teams_dict

In [23]:
# Initialize the Elo ratings for each team
elo_ratings = init_elos(data)

# Calculate the Elo ratings for each team
data, elo_ratings = calculate_elo_ratings(data, elo_ratings)

/var/folders/bl/xbg_536n2_n123s38z4hxbx00000gn/T/ipykernel_68337/388861480.py:29: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_list = (df['pt1'].append(df['pt2'])).unique()


In [26]:
data.head()

,pt1,pt2,score_pt1,score_pt2,ground,country,league,date,time,season,...,crosses_pt2,dribbles_completed_pt2,dribbles_pt2,possession_lost_pt2,duels_won_pt2,aerials_won_pt2,tackles_pt2,interceptions_pt2,clearances_pt2,attendance
0,palermo,lazio,5,1,Renzo Barbera,Italy,Italian Serie A,2012-02-19 19:45:00+00:00,19:45:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
1,novara,atalanta,0,0,Silvio Piola,NaN,Italian Serie A,2012-02-19 14:00:00+00:00,14:00:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
2,udinese,cagliari,0,0,Friuli,Italy,Italian Serie A,2012-02-19 19:45:00+00:00,19:45:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
3,lecce,siena,4,1,Via del Mare,Italy,Italian Serie A,2012-02-19 11:30:00+00:00,11:30:00,2012,...,0,0,0,0,0,0,0,0,0,NaN
4,as_roma,parma,1,0,Olimpico,NaN,Italian Serie A,2012-02-19 14:00:00+00:00,14:00:00,2012,...,0,0,0,0,0,0,0,0,0,NaN


In [30]:
data['has_stats'].value_counts()

False    35259
True      1978
Name: has_stats, dtype: int64